### 1. ain-py 설치
AI Network를 이용한 작업을 하기 위해 ain-py 패키지를 설치해줍니다.

nest_asyncio는 노트북 환경에서 비동기 작업시 발생하는 에러를 해결하기 위해 설치해줍니다.

In [1]:
!pip install ain-py
!pip install nest_asyncio

### 2. 패키지 import

In [2]:
import asyncio
import nest_asyncio
from time import time
from ain.ain import Ain
from ain.types import ValueOnlyTransactionInput

nest_asyncio.apply()

loop = asyncio.get_event_loop()

### 3. AI Network wallet 생성
AI Network wallet을 생성하고, default wallet을 지정해줍니다.

In [3]:
ain = Ain("https://testnet-api.ainetwork.ai", chainId=None)

# Create address and Set default account
accounts = ain.wallet.create(1)
my_address = accounts[0]
ain.wallet.setDefaultAccount(my_address)

### 4. App 생성
App의 이름이 겹치지 않기 위해 App의 이름에 timestamp가 포함되도록 합니다.

그 후 App의 경로를 설정한 후, SET_VALUE 작업을 통해 App의 /manage_app/{appName}/create/{key} 경로에 owner를 설정합니다.

Owner가 설정되면, /apps/{appName} 경로에 대한 모든 owner 권한(branch_owner, write_function, write_owner, write_rule)을 가집니다.

이 과정에서 트랜잭션이 발생합니다.

In [4]:
# Use your own app name
app_name = f"my_app_{int(time()*1000)}"
app_path = f"/apps/{app_name}"

# Create an app at /apps/{appName}. With the admin config below,
# only 'my_address' will have owner & write permissions at /apps/{app_name}.
async def create_my_app():
    result = await ain.db.ref(
        f'/manage_app/{app_name}/create/{int(time()*1000)}'
    ).setValue(
        ValueOnlyTransactionInput(
            value={
                'admin': {
                    my_address: True
                }
            },
            nonce=-1
        )
    )
    print(result)
loop.run_until_complete(create_my_app())

{'tx_hash': '0x30a9b45c57e3dc58edcefbe10ed42ad3bb0f2d4e0f488a784e893f2f88d3bc74', 'result': {'gas_amount_total': {'bandwidth': {'service': 2, 'app': {'my_app_1641880649312': 2}}, 'state': {'service': 1566}}, 'gas_cost_total': 0, 'func_results': {'_createApp': {'op_results': {'0': {'path': '/apps/my_app_1641880649312', 'result': {'code': 0, 'bandwidth_gas_amount': 1}}, '1': {'path': '/apps/my_app_1641880649312', 'result': {'code': 0, 'bandwidth_gas_amount': 1}}, '2': {'path': '/manage_app/my_app_1641880649312/config', 'result': {'code': 0, 'bandwidth_gas_amount': 1}}}, 'code': 0, 'bandwidth_gas_amount': 0}}, 'code': 0, 'bandwidth_gas_amount': 1, 'gas_amount_charged': 1568}}


### 5. Transaction 보내기
/apps/{appName} 하위 경로에 대해서 SET_VALUE를 통해 값을 작성합니다.

이 과정에서 트랜잭션이 발생합니다.

In [5]:
# Write value to user_message_path.
async def set_value():
    user_message_path = f'{app_path}/messages/{my_address}'
    result = await ain.db.ref(
        f'{user_message_path}/{int(time()*1000)}/user'
    ).setValue(
        ValueOnlyTransactionInput(value='Hello AI Network', nonce=-1)
    )
    print(result)
loop.run_until_complete(set_value())

{'tx_hash': '0x920bc14c3cf2f1faf994b478911d4d022b05e42b823124dcacb57fcdcc68e74d', 'result': {'gas_amount_total': {'bandwidth': {'service': 0, 'app': {'my_app_1641880649312': 1}}, 'state': {'service': 0, 'app': {'my_app_1641880649312': 966}}}, 'gas_cost_total': 0, 'code': 0, 'bandwidth_gas_amount': 1, 'gas_amount_charged': 0}}
